# Data Extraction

merge old-format data and new-format data

in result:
- set of csv files per year: sales
- set of csv files per year: balance

sales file:
- date: "YYYY-mm-DD"
- id
- name
- sales

balance file:
- date: "YYYY-mm-DD"
- id
- name
- balance

### importing

In [103]:
import sys
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from pandas import read_csv

### read old data 

In [301]:
def removeHeaderRows(data_raw):
    data_raw.drop(data_raw.index[[0,1,2,3,4]], inplace=True)
        
def removeSingleValueColumn(col_name, data):
    if (len(data[col_name].unique()) == 1):
        print('remove column: ', col_name)
        return data.drop(col_name, 1)
    
#read raw data
from pandas import read_csv
data_raw2014 = read_csv("~/Documents/imagineML/data/data2014.csv") #щука
data_raw2015 = read_csv("~/Documents/imagineML/data/data2015.csv")
data_raw2016 = read_csv("~/Documents/imagineML/data/data2016.csv")
data_raw2017 = read_csv("~/Documents/imagineML/data/data2017.csv")
totalrows = data_raw2014.shape[0] + data_raw2015.shape[0] + data_raw2016.shape[0] + data_raw2017.shape[0]
print("~rows: ",totalrows)

removeHeaderRows(data_raw2014)
removeHeaderRows(data_raw2015)
removeHeaderRows(data_raw2016)
removeHeaderRows(data_raw2017)

df = pd.concat([data_raw2014, data_raw2015, data_raw2016, data_raw2017], axis=0)
df = df.reset_index(drop=True)
print("actual rows: ", df.shape[0])

df.columns = ['date', 'col1', 'warehouse', 'col2', 'name', 'col3', 'id', 'qtyBefore', 'qty']
df = removeSingleValueColumn('col1', df)
df = removeSingleValueColumn('col2', df)
df = removeSingleValueColumn('col3', df)
df = removeSingleValueColumn('warehouse', df)
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
df['date'] = df['date'].dt.date
df['year'] = df['date'].map(lambda x: x.year)
df['qtyBefore'] = df['qtyBefore'].astype('float')
df['qty'] = df['qty'].astype('float')
df.head()
data_old = df

~rows:  96579
actual rows:  96559
remove column:  col1
remove column:  col2
remove column:  col3
remove column:  warehouse


### read new data

In [303]:
df = read_csv("~/Documents/imagineML/data/result.csv",sep=';')

In [304]:
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.date
df['year'] = df['date'].map(lambda x: x.year)
data_new = df

### create map keys of items

In [275]:
all_names = data_old['name'].append(data_new['name'])
print("check", len(data_old) + len(data_new), len(all_names))
all_names = all_names.sort_values().unique()

check 925276 925276


In [276]:
keys = pd.DataFrame({"item_name": all_names })
keys.index += 1
keys.head()

,item_name
1,100 причин моей любви
2,"1940 DVD-открытка ""Ты родился"" 1940"
3,"1941 DVD-открытка ""Ты родился"" 1941"
4,"1942 DVD-открытка ""Ты родился"" 1942"
5,"1943 DVD-открытка ""Ты родился"" 1943"


In [277]:
keys_dict = keys
keys_dict['item_key'] = keys.index
keys_dict.set_index('item_name', inplace=True)
keys_dict.head()

,item_key
item_name,
100 причин моей любви,1
"1940 DVD-открытка ""Ты родился"" 1940",2
"1941 DVD-открытка ""Ты родился"" 1941",3
"1942 DVD-открытка ""Ты родился"" 1942",4
"1943 DVD-открытка ""Ты родился"" 1943",5


In [278]:
def get_item_key(item_name):
    return keys_dict.loc[item_name].item_key

In [306]:
data_old['key'] = data_old['name'].map(lambda x: get_item_key(x))
data_new['key'] = data_new['name'].map(lambda x: get_item_key(x))

### merge sales data 

In [307]:
sales_old = data_old[['key', 'name', 'date', 'qty', 'year']]
sales_old.rename(columns={'qty': 'sales'}, inplace=True)
sales_old.head()

,key,name,date,sales,year
0,262,Батарейки мизинчиковые ААА LR03 12 в уп. бата...,2014-07-11,3.0,2014
1,409,Блокнотик Светская львица,2014-07-11,1.0,2014
2,679,Визитница Для покупочек,2014-07-11,1.0,2014
3,685,Визитница Мои скидочки,2014-07-11,1.0,2014
4,688,Визитница Особы приближенные к императору,2014-07-11,1.0,2014


In [316]:
sales_new = data_new[['key', 'name', 'date', 'sales', 'year']]
sales_new = sales_new.sort_values(['date','name'],ascending=[True,True])
sales_new.head()

,key,name,date,sales,year
178,7,"1945 DVD-открытка ""Ты родился"" 1945",2017-08-01,0.0,2017
825287,7,"1945 DVD-открытка ""Ты родился"" 1945",2017-08-01,0.0,2017
3084,12,"1950 DVD-открытка ""Ты родился"" 1950",2017-08-01,0.0,2017
828193,12,"1950 DVD-открытка ""Ты родился"" 1950",2017-08-01,0.0,2017
3083,13,"1951 DVD-открытка ""Ты родился"" 1951",2017-08-01,0.0,2017


In [317]:
sales = pd.concat([sales_old, sales_new],axis=0)
print('check', len(sales), len(sales_old)+len(sales_new))
sales = sales.sort_values(['date', 'key'])

check 925276 925276


In [313]:
years = sales['year'].unique()
for year in years:
    sales_file = '~/Documents/imagineML/data/sales%s.csv'%year
    sales[sales['year'] == year][['key','name','date','sales']].to_csv(sales_file, sep='\t', encoding='utf-8')

### merge balance data

the same as sales data

In [321]:
balance_old = data_old[['key', 'name', 'date', 'qtyBefore', 'year']]
balance_old.rename(columns={'qtyBefore': 'balance'}, inplace=True)
balance_new = data_new[['key','name','date','balance', 'year']]
balance = pd.concat([balance_old, balance_new],axis=0)
balance = balance.sort_values(['date','name'],ascending=[True,True])
print('check', len(balance), len(balance_old)+len(balance_new))

check 925276 925276


In [322]:
for year in years:
    balance_file = '~/Documents/imagineML/data/balance%s.csv'%year
    balance[balance['year'] == year][['key','name','date','balance']].to_csv(balance_file, sep='\t', encoding='utf-8')

In [324]:
keys.to_csv('~/Documents/imagineML/data/itemkeys.csv', sep='\t', encoding='utf-8')